In [35]:
import numpy as np

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [36]:
def calculate_performance_metrics(X_test, y_true, model):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_true))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        macro_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='macro')
        weighted_auc = roc_auc_score(label_binarize(all_labels, classes=range(num_classes)), probabilities[:,1], average='weighted')

        cm = confusion_matrix(all_labels, all_preds)

        def calculate_class_metrics_macro(cm, class_index):
            TP = cm[class_index, class_index]
            FP = cm[:, class_index].sum() - TP
            FN = cm[class_index, :].sum() - TP
            TN = cm.sum() - (TP + FP + FN)
            
            TPR = TP / (TP + FN) if (TP + FN) != 0 else 0  
            TNR = TN / (TN + FP) if (TN + FP) != 0 else 0  
            FPR = FP / (FP + TN) if (FP + TN) != 0 else 0  
            FNR = FN / (FN + TP) if (FN + TP) != 0 else 0  
            
            return TPR, TNR, FPR, FNR
            
        metrics = np.array([calculate_class_metrics_macro(cm, i) for i in range(num_classes)])
        TPR_macro, TNR_macro, FPR_macro, FNR_macro = np.mean(metrics, axis=0)

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}\nAUC: {macro_auc}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}\nAUC: {weighted_auc}")
        print()
        
        print(f"Mean FNR: {FNR_macro}\nMean TNR: {TNR_macro}\nMean FPR: {FPR_macro}\nMean TPR: {TPR_macro}")


In [37]:
x_train = np.load("/home/jovyan/UNSW/x_train.npy")
y_train = np.load("/home/jovyan/UNSW/y_train.npy")
x_val = np.load("/home/jovyan/UNSW/x_val.npy")
y_val = np.load("/home/jovyan/UNSW/y_val.npy")
x_test = np.load("/home/jovyan/UNSW/x_test.npy")
y_test = np.load("/home/jovyan/UNSW/y_test.npy")

In [38]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [39]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [40]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [41]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [42]:
for epoch in range(50):
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    model.eval()
    val_train_loss = 0.0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            val_train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            correct_predictions += (predicted == labels).sum().item()

    avg_val_loss = val_train_loss / len(val_loader)
    val_accuracy = correct_predictions / len(val_dataset)

    print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping check using min_delta
    if best_loss - avg_val_loss > min_delta:
        best_loss = avg_val_loss
        patience_counter = 0
    else:
        patience_counter += 1

    if patience_counter >= patience:
        print("Early stopping triggered")
        break

Epoch 1, Training Loss: 0.2146, Validation Loss: 0.1386, Validation Accuracy: 0.9462
Epoch 2, Training Loss: 0.1309, Validation Loss: 0.1184, Validation Accuracy: 0.9535
Epoch 3, Training Loss: 0.1171, Validation Loss: 0.1143, Validation Accuracy: 0.9528
Epoch 4, Training Loss: 0.1104, Validation Loss: 0.1033, Validation Accuracy: 0.9603
Epoch 5, Training Loss: 0.1048, Validation Loss: 0.1092, Validation Accuracy: 0.9539
Epoch 6, Training Loss: 0.1014, Validation Loss: 0.0943, Validation Accuracy: 0.9622
Epoch 7, Training Loss: 0.1001, Validation Loss: 0.0947, Validation Accuracy: 0.9628
Epoch 8, Training Loss: 0.0974, Validation Loss: 0.1002, Validation Accuracy: 0.9584
Epoch 9, Training Loss: 0.0962, Validation Loss: 0.0932, Validation Accuracy: 0.9619
Epoch 10, Training Loss: 0.0945, Validation Loss: 0.0894, Validation Accuracy: 0.9657
Epoch 11, Training Loss: 0.0930, Validation Loss: 0.0919, Validation Accuracy: 0.9636
Epoch 12, Training Loss: 0.0923, Validation Loss: 0.0873, Valid

In [43]:
calculate_performance_metrics(x_test, y_test, model)

Accuracy: 0.9202077721525651

macro
Precision: 0.9127761855701064
Recall: 0.9019262445280545
F1 Score: 0.9070354096786635
AUC: 0.9668989620072981

weighted
Precision: 0.9196316414457625
Recall: 0.9202077721525651
F1 Score: 0.9196765913623771
AUC: 0.9668989620072981

Mean FNR: 0.09807375547194551
Mean TNR: 0.9019262445280545
Mean FPR: 0.09807375547194551
Mean TPR: 0.9019262445280545


In [44]:
torch.save(model.state_dict(), "/home/jovyan/UNSW/transfer_attack/dnn_pytorch.pt")

In [45]:
model1 = DNNModel(input_size=input_shape, output_size=output_shape).to(device)
model1.load_state_dict(torch.load("/home/jovyan/UNSW/transfer_attack/dnn_pytorch.pt"))
calculate_performance_metrics(x_test, y_test, model1)

Accuracy: 0.9202077721525651

macro
Precision: 0.9127761855701064
Recall: 0.9019262445280545
F1 Score: 0.9070354096786635
AUC: 0.9668989620072981

weighted
Precision: 0.9196316414457625
Recall: 0.9202077721525651
F1 Score: 0.9196765913623771
AUC: 0.9668989620072981

Mean FNR: 0.09807375547194551
Mean TNR: 0.9019262445280545
Mean FPR: 0.09807375547194551
Mean TPR: 0.9019262445280545
